# Linux shell commands using Bash

In [ ]:
# Two numbers should be produced and they should be identical
commands="tree find ls xxd cat head tail wc file fgrep cut seq shuf sed awk mv mkdir cp rm rmdir"
<<< ${commands} tr ' ' '\n' | wc -l
which ${commands} | wc -l

## Files and the Filesystem

Computer operating systems use a number of different data structures in order to operate.  We will start exploring two of those data structures: the filesystem and files.

Using [the CRUD model]( https://en.wikipedia.org/wiki/Create,_read,_update_and_delete ), we will start with the filesystem then later we'll explore files.  And we are going to start by using two "Read" commands: `tree` and `find`.


### tree and find commands

The filesystem is a hierarchy of files and directories ( aka folders ).
Folders can contain both files and folders ( called subfolders or subdirectories ).
The commands `tree` and `find` allow us to traverse ( aka descend or recurse ) this hierarchy.

> Note: As a shorthand, when I write `file`, I will often mean both files and folders.

Starting from a specified location, both commands will show all the files that are "lower" in the hierarchy.

In these two examples, I am specifying the starting location as `/etc/apt`.

In [ ]:
tree /etc/apt

Those are all the files that are "below" `/etc/apt`.

The `tree` command provides a nice, graphic representation of the hierarchy, 
with lower levels indented from upper levels and connected by lines.
We can distinguish between files and folders by using the `-F` flag ( aka option ), which adds a `/` to the end of folders.  

In [ ]:
tree -F /etc/apt

Notice that all the folders end with a `/`, but files do not.

In contrast to the `tree` command, the `find` command provides a **listing** of all the files in the heirarchy. 

In [ ]:
find /etc/apt

There's no easy equivalent option to `-F` with `find`.


### paths

The `/etc/apt/` notation specifies what is known as a `path`, which is a sequence of folder names separated by the `/` character.  For example, we can go a level deeper by appending `trusted.gpg.d` to the path.


In [ ]:
tree -F /etc/apt/trusted.gpg.d

### ls command

`tree` and `find` are not the only ways to traverse and display the filesystem hierarchy. 
`ls` is most commonly used to display the contents of a single folder.  And like `tree` it can use the `-F` option to append a `/` to folders.

In [ ]:
ls -F /etc/apt/

`ls` can also traverse the filesystem hierarchy, but I find the output a bit more challenging to interpret

In [ ]:
ls -FR /etc/apt/

`ls` can not only provide us with a listing of the contents of a folder, that is, a list of the names of files and folders.  It can also display meta-data, that is, information about the files and folders such as permissions, ownership, membership, size, and dates, when combined with the `-l` option.

In [ ]:
ls -Fl /etc/apt/

To use this entry as an example:
```
-rw-r--r-- 1 root root 2765 Jan 26 08:32 sources.list
```

- `-rw-r--r--` are the permissions
- `1` the reference count
- `root` the owner
- `root` the group membership
- `2765` the size
- `Jan 26` the date of last modification
- `08:32` the time of last modification
- `sources.list` the name of the file




Within a folder there often "hidden" files and folders.  Files that begin with a dot ( "." ) are not displayed unless the `-a` option is used with `ls`.

In [ ]:
ls -Fla /etc/apt/

Notice that we now see two additional folders: ./ and ../.  Those are actually "pointer" folders in that they point to the current folder ( ./ ) and to the parent folder ( ../ ).

There is much more to mention about the file system, in particular the remaing CRUD operations.  But we'll change our focus to files and return to the file system later.

## Files

### File contents

#### cat

Just like `ls` displays the contents of a folder, `cat` displays the contents of a file ... with some interpretation.  We'll use the file ` /etc/debian_version` as an example.

In [ ]:
cat /etc/debian_version

#### xxd

`cat` displays the characters that are in the file. But characters are not what is really stored on the filesystem.  Rather computers store everything as numbers, specifically 1's and 0's.  And we can display that using the `xxd` command.

In [ ]:
xxd -b -g0 /etc/debian_version

We see three fields.  The field on the left is the address offset, ending in a colon ( : ). Then we see a series of 1's and 0's.  Those are called bits from "binary digits."  Those bits are the actual content of the file. On the right, we see the character interpretation of those 1's and 0's.

Here's the same output but where the bits have been put into groups of 8 called bytes.

In [ ]:
xxd -b -g1 /etc/debian_version

On the far right of the output, we see that the first group of 8 bits ( the first byte ) gets interpreted as the letters 'b', the second as letter 'u', etc.  Also notice that the last byte '00001010' is being displayed as a dot ( '.' ). That byte is actually a non-printable character, one of many.  This one happens to be the end-of-line ( aka newline or '\n' ) character, which we will encounter more of later on.  Other non-printable characters frequently encountered include tab ( '\t' ) , carriage return ( '\r' ), and null ( '\0' ).

As an alternative to binary, we can display the bytes in a file in hexadecimal by using the `-u` option.

In [ ]:
xxd -u -g1 /etc/debian_version

We see that hex 62 maps to 'b', hex 75 to 'u', etc.

`od` is another tool to display the contents of a file.

In [ ]:
od -t x1 -c /etc/debian_version

### Lines

The interpretation of the bytes in a file is left to programs.  




One way to interpret a file stream is as a collection of "lines" with each line being a collection of "text" characters.  Many files use this approach, e.g. CSV, YAML, HTML.  For these "text" files, a program can read the file one character at a time until it gets to an "end-of-line" character, then it can operate on that line, then read the next line.  `cat` does this with every line in a file. For example, we can have `cat` prefix each line with the line number:

In [ ]:
cat -n /etc/debian_version

That's not very exciting with a file that has only one line.  So, here's the same command run on a file with multiple lines:

In [ ]:
cat -n /etc/os-release

In this case, `cat` reads each character until it gets to the end of the line, prints the line number followed by the line, then repeats the process until it gets to the end of the file.

Many other commands use this pattern of reading a line, operate on it, then repeat.  Let's look at a few:
- head
- tail
- cut
- wc
- file

BTW, for all these commands, much more details on options and how they work can be found using a Google search for "unix man " followed by the command.  For example, "[unix man head](https://www.google.com/search?q=unix+man+head)"

In [ ]:
# head displays the first 10 lines of a file if not given any options
## you can specify more or fewer lines by giving it the option -n X, where X is a whole number
head -n 4 /etc/os-release

In [ ]:
# tail displays the last 10 lines of a file if not given any options
## you can specify more or fewer lines by giving it the option -n X, where X is a whole number
tail -n 4 /etc/os-release

In [ ]:
# cut displays the character range specified by the -c option or a field range specified by the -f option
## range is specified using 1-based counting
## here we get the first 6 characters from the /etc/debian_version file.
cat /etc/debian_version
cut -c 1-6 /etc/debian_version

In [ ]:
## here we get characters 7-10 from the /etc/debian_version file.
cut -c 7-10 /etc/debian_version

In [ ]:
# wc gives a summary of how many lines, words, and characters there are in a file
wc /etc/os-release

In [ ]:
# file gives you a reasonable guess as to what type of file it is.
file /etc/debian_version
file /etc/os-release
file /bin/grep

The caveat is that these programs only work if the file is organized as a "text" file.  That is, the bytes are interpreted as alpha-numeric characters with line endings.  When some other convention is used, then the file is termed a "binary" file. This can lead to some confusion as all files are "binary".  The difference is in how the bytes are organized in the file and interpreted by some program.

## Creating a file

The previous examples used pre-existing files.  Now we will use some commands that will create data and then put them into a file. We will explore the following commands:
- date
- echo
- seq
- curl

In [ ]:
# date prints the date
date

We can tell a command to put the data into a file by redirecting its output.  That is done using the '>' symbol.  For example, to save the output from the `date` to a file called `date.txt`:

In [ ]:
# show that date.txt does not exist
ls -F

In [ ]:
# generate a date and redirect the output to the date.txt file
date > date.txt


In [ ]:
# show that date.txt now does exist
ls -F

In [ ]:
# display the contents of the date.txt file with a line number
cat -n date.txt

In [ ]:
# echo displays the provided text
echo 'Hello, world!'

In [ ]:
# to save output to a file
echo 'Hello, world!' > hw.txt

In [ ]:
# display the contents
cat -n hw.txt

In [ ]:
# seq generates a range of numbers
seq 1 10 > seq.txt
cat -n seq.txt

In [ ]:
# curl GETs a webpage
## curl downloads a file containing air quality data from the city of Albuquerque
curl -s http://data.cabq.gov/airquality/aqindex/history/042222.0017 > abq.air-quality.dat
wc abq.air-quality.dat
head abq.air-quality.dat


## Command pipeline

Much like one can do method chaining in Python, Ruby, JavaScript, and other languages, commands can be piped together using a vertical bar '|'.  In this way, the output of one command can be piped as input into the next command.  For example:

In [ ]:
# here the first ten lines of a file are numbered
head abq.air-quality.dat | cat -n

In [ ]:
# here only the first field is displayed from the first ten lines and then numbered
head abq.air-quality.dat | cut -d, -f 1 | cat -n

In [ ]:
cat -n abq.air-quality.dat | shuf -n 10

In [ ]:
# randomly select 100 words 
shuf -n 100 /usr/share/dict/words | cut -c1-10 | head | cat -n


In [ ]:
file bash.intro.ipynb

In [ ]:
jq .cells[1].source bash.intro.ipynb